In [117]:
#Bin Packing problem
#Cutting stock problem
def BinPackingExamples():
    """
    Generate a bin packing problem example.
    input : none
    ouput :
        s : the set of items.
        B : the capacity of bins.
    """
    B = 9
    w = [2,3,4,5,6] #[2,3,4,5,6,7,8] #アイテムの大きさ
    q = [1]*len(w) #[4,2,6,6,2,2,2] #アイテムの必要量
    s=[]
    for j in range(len(w)):
        for i in range(q[j]):
            s.append(w[j])
    return s, w, q, B

s, w, q, B = BinPackingExamples()
print("items : ", s, "\n the requirement number of items : ", q, "\n the capacity of bins : ", B)

items :  [2, 3, 4, 5, 6] 
 the requirement number of items :  [1, 1, 1, 1, 1] 
 the capacity of bins :  9


In [118]:
#1. 各種類のアイテムを1種類だけ使うというパターンを生成．
t=[]
m = len(w)
for i in range(m):
    pat = [0]*m
    pat[i] = int(B/w[i])
    t.append(pat)

from scipy.optimize import linprog
import gurobi as gp
A_d = t
new_pattern = []
while(1):
    #2. 双対問題を解く
    A_d = A_d + [new_pattern]*(new_pattern!=[])
    b_d = [1]*len(A_d)
    c_d = list(map(lambda x: -x, q)) #使うモジュールの関係で目的関数が必ずminになる
    ans_d = linprog(c_d, A_ub=A_d, b_ub=b_d, method="simplex")
    #print(ans_d, "\n\n")

    #3. ナップサック問題をとく
    A_kp = w
    b_kp = B
    c_kp = list(ans_d.x)
    Idx = list(range(len(A_kp)))
    w_per_v = {idx:weight/value for idx,value,weight in zip(Idx, A_kp, c_kp)}
    w_per_v = sorted(w_per_v.items(), key=lambda x:x[1], reverse=True)
    ans_kp = {i:0 for i in range(len(w))}
    for item in w_per_v:
        if b_kp >= 0:
            ans_kp[item[0]] = b_kp//w[item[0]]
            b_kp -= ans[item[0]]*w[item[0]]
    obj_kp = 0
    for i in range(len(ans_kp)):
        obj_kp += ans_kp[i]*c_kp[i]
        
    ##3.1. ナップサック問題の最適解が1よりも大きい
    ###3.1.1. 双対問題に新たな制約を加える
    if obj_kp > 1:
        new_pattern = list(ans_kp.values())
    ###3.1.2. 2.へ

    ##3.2. ナップサック問題の最適解が1以下　→　終了
    else:
        #print(A_d)
        print("双対緩和問題の解 : \n", ans_d)
        
        model = gp.Model()
        y = {i:model.addVar(lb = 0, ub = 1, vtype = gp.GRB.BINARY, name = "y"+str(i)) for i in range(len(ans_d.x))}
        model.setObjective(gp.quicksum(q[i]*y[i] for i in range(len(ans_d.x))), sense=gp.GRB.MAXIMIZE)
        for idx, constraint in enumerate(A_d):
            model.addConstr(gp.quicksum(constraint[i]*y[i] for i in range(len(constraint))) <= 1, name="constraint"+ str(idx))
        model.optimize()
        break

双対緩和問題の解 : 
      con: array([], dtype=float64)
     fun: -2.0
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([1., 1., 1., 0., 0., 0., 0.])
  status: 0
 success: True
       x: array([0., 0., 0., 1., 1.])
Using license file /Users/okamoto/gurobi.lic


GurobiError: Version number is 9.0, license is for version 8.0

In [58]:
#Initial set of cutting pattern
#各種類のアイテムを1種類だけ使うというパターンを生成．
t=[]
m = len(w)
for i in range(m):
    pat = [0]*m
    pat[i] = int(B/w[i])
    t.append(pat)
    
print("Initial Cutting Pattern : \n", t)

Initial Cutting Pattern : 
 [[4, 0, 0, 0, 0, 0, 0], [0, 3, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1]]


In [59]:
#双対問題を解く
A_d = t
b_d = [1]*len(A_d)
c_d = [-1]*len(A_d[1]) #使うモジュールの関係で目的関数が必ずminになる

#print(E, "\n", constraints_lval, "\n", constraints_rval, "\n", obj_coefficient)

print(A_d, "\n", b_d, "\n", c_d)

[[4, 0, 0, 0, 0, 0, 0], [0, 3, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1]] 
 [1, 1, 1, 1, 1, 1, 1] 
 [-1, -1, -1, -1, -1, -1, -1]


In [60]:
from scipy.optimize import linprog
ans_d = linprog(c_d, A_ub=A_d, b_ub=b_d, method="simplex")
ans_d

     con: array([], dtype=float64)
     fun: -5.083333333333333
 message: 'The solution was determined in presolve as there are no non-trivial constraints.'
     nit: 0
   slack: array([0., 0., 0., 0., 0., 0., 0.])
  status: 0
 success: True
       x: array([0.25      , 0.33333333, 0.5       , 1.        , 1.        ,
       1.        , 1.        ])

# ナップサック問題を解いて，良いパッキングを見つける

In [61]:
#ナップサック問題をとく
A_kp = w
b_kp = B
c_kp = list(ans_d.x)#list(map(lambda x: -x, list(ans.x)))

print(A_kp, "\n", b_kp, "\n", c_kp)

[2, 3, 4, 5, 6, 7, 8] 
 9 
 [0.25, 0.3333333333333333, 0.5, 1.0, 1.0, 1.0, 1.0]


In [62]:
Idx = list(range(len(A_kp)))
w_per_v = {idx:weight/value for idx,value,weight in zip(Idx, A_kp, c_kp)}
#print(w_per_v)
w_per_v = sorted(w_per_v.items(), key=lambda x:x[1], reverse=True)
#print(w_per_v)
ans_kp = {i:0 for i in range(len(w))}
for item in w_per_v:
    if b_kp >= 0:
        ans_kp[item[0]] = b_kp//w[item[0]]
        b_kp -= ans[item[0]]*w[item[0]]
print(ans_kp)

{0: 2, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0}


In [63]:
#ナップサック問題の最適解が1よりも大きいか
obj_kp = 0
for i in range(len(ans_kp)):
    obj_kp += ans_kp[i]*c_kp[i]
print(obj_kp)

1.5


In [64]:
#双対問題に新たな制約を加える
if obj_kp > 1:
    A_d.append(list(ans_kp.values()))
    b_d = [1]*len(A_d)
    c_d = [-1]*len(A_d[1]) #使うモジュールの関係で目的関数が必ずminになる

    #print(E, "\n", constraints_lval, "\n", constraints_rval, "\n", obj_coefficient)

    print(A_d, "\n", b_d, "\n", c_d)

[[4, 0, 0, 0, 0, 0, 0], [0, 3, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1], [2, 0, 0, 1, 0, 0, 0]] 
 [1, 1, 1, 1, 1, 1, 1, 1] 
 [-1, -1, -1, -1, -1, -1, -1]


In [66]:
ans_d = linprog(c_d, A_ub=A_d, b_ub=b_d, method="simplex")
ans_d

     con: array([], dtype=float64)
     fun: -4.833333333333333
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([1., 0., 0., 0., 0., 0., 0., 0.])
  status: 0
 success: True
       x: array([0.        , 0.33333333, 0.5       , 1.        , 1.        ,
       1.        , 1.        ])